In [114]:
pip install ucimlrepo


Note: you may need to restart the kernel to use updated packages.


In [115]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns


In [116]:
import sklearn
print(sklearn.__version__)


1.4.2


In [117]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
y = car_evaluation.data.targets 
  
# metadata 
print(car_evaluation.metadata) 
  
# variable information 
print(car_evaluation.variables) 


{'uci_id': 19, 'name': 'Car Evaluation', 'repository_url': 'https://archive.ics.uci.edu/dataset/19/car+evaluation', 'data_url': 'https://archive.ics.uci.edu/static/public/19/data.csv', 'abstract': 'Derived from simple hierarchical decision model, this database may be useful for testing constructive induction and structure discovery methods.', 'area': 'Other', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1728, 'num_features': 6, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5JP48', 'creators': ['Marko Bohanec'], 'intro_paper': {'ID': 249, 'type': 'NATIVE', 'title': 'Knowledge acquisition and explanation for multi-attribute decision making', 'authors': 'M. Bohanec, V. Rajkovič', 'venue': '8th Intl Workshop on Expert Systems and their Applications, 

In [118]:
# %%
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
dtypes: object(6)
memory usage: 81.1+ KB


In [119]:
y['class'].value_counts()

class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64

In [120]:
y['class'] = np.where(y['class'] == 'unacc',0,1)

C:\Users\dai\AppData\Local\Temp\ipykernel_12828\1808155609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['class'] = np.where(y['class'] == 'unacc',0,1)


In [121]:
y['class'].value_counts(normalize=True)*100 # normalise = True ) * 100 for converting into percentage

class
0    70.023148
1    29.976852
Name: proportion, dtype: float64

In [122]:
# train test Split

X_train,X_test,y_train,y_test = train_test_split(X,y, random_state = 24,test_size = 0.3,stratify=y['class'])
# stratify -> after this results should improve if not that's coincidence
print(y_train['class'].value_counts(normalize = True)*100)
print(y_test['class'].value_counts(normalize = True)*100)

class
0    70.057899
1    29.942101
Name: proportion, dtype: float64
class
0    69.942197
1    30.057803
Name: proportion, dtype: float64


In [123]:
ohe = OneHotEncoder(sparse_output = False, drop = 'first',handle_unknown='ignore').set_output(transform = 'pandas') 

# to change it into pandas

In [124]:
X_trn_ohe = ohe.fit_transform(X_train)
X_tst_ohe = ohe.transform(X_test)

In [125]:
lr = LogisticRegression()

In [126]:
lr.fit(X_trn_ohe,y_train['class'])
y_pred = lr.predict(X_tst_ohe)
print(accuracy_score(y_test['class'],y_pred))

0.9633911368015414


In [127]:
lr = LogisticRegression(solver='liblinear')

In [128]:
lr.fit(X_trn_ohe,y_train['class'])
y_pred = lr.predict(X_tst_ohe)
print(accuracy_score(y_test['class'],y_pred))

0.9595375722543352


In [129]:
lr = LogisticRegression(solver='newton-cholesky')

In [130]:
lr.fit(X_trn_ohe,y_train['class'])
y_pred = lr.predict(X_tst_ohe)
print(accuracy_score(y_test['class'],y_pred))

0.9633911368015414


In [131]:
lr = LogisticRegression(solver='lbfgs',penalty = None) # only works with penalty = None

In [132]:
lr.fit(X_trn_ohe,y_train['class'])
y_pred = lr.predict(X_tst_ohe)
print(accuracy_score(y_test['class'],y_pred))

0.9653179190751445


In [133]:
# K- Flds CV

In [134]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline



In [135]:
ohe = OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse_output = False)

In [136]:
lr = LogisticRegression(multi_class = 'multinomial')
pipe = Pipeline([('OHE',ohe),('LR',lr)])
params = {'LR__solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga']}

In [137]:
gcv = GridSearchCV(pipe, param_grid = params)
gcv.fit(X,y)

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-v

GridSearchCV(estimator=Pipeline(steps=[('OHE',
                                        OneHotEncoder(drop='first',
                                                      handle_unknown='ignore',
                                                      sparse_output=False)),
                                       ('LR',
                                        LogisticRegression(multi_class='multinomial'))]),
             param_grid={'LR__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                        'newton-cholesky', 'sag', 'saga']})

In [138]:
print(gcv.best_params_)
print(gcv.best_score_)

{'LR__solver': 'lbfgs'}
0.8303409566892854


In [139]:
lr = LogisticRegression(multi_class = 'ovr')
pipe = Pipeline([('OHE',ohe),('LR',lr)])
params = {'LR__solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga']}

In [140]:
gcv = GridSearchCV(pipe, param_grid = params)
gcv.fit(X,y)

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-v

GridSearchCV(estimator=Pipeline(steps=[('OHE',
                                        OneHotEncoder(drop='first',
                                                      handle_unknown='ignore',
                                                      sparse_output=False)),
                                       ('LR',
                                        LogisticRegression(multi_class='ovr'))]),
             param_grid={'LR__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                        'newton-cholesky', 'sag', 'saga']})

In [141]:
print(gcv.best_params_)
print(gcv.best_score_)

{'LR__solver': 'newton-cg'}
0.829761246544358


In [184]:
#multi class [ovr,multinomial]

ohe = OneHotEncoder(drop = 'first', handle_unknown = 'ignore', sparse_output = False)

In [186]:
lr = LogisticRegression(multi_class = 'multinomial',random_state =24)
kfold = StratifiedKFold(n_splits = 5, random_state =24, shuffle = True)
pipe = Pipeline([('OHE',ohe),('LR',lr)])
params = {'LR__solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga'],
         'LR__multi_class':['ovr','multinomial'],
         'LR__C':np.linspace(0.001,10,20)}

In [194]:
gcv = GridSearchCV(pipe, param_grid = params,cv = kfold)
gcv.fit(X,y['class'])

C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.w

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('OHE',
                                        OneHotEncoder(drop='first',
                                                      handle_unknown='ignore',
                                                      sparse_output=False)),
                                       ('LR',
                                        LogisticRegression(multi_class='multinomial',
                                                           random_state=24))]),
             param_grid={'LR__C': array([1.00000000e-03, 5.27263158e-01, 1.05352632e+00, 1.57978947e+00,
       2.10605263e+00, 2.63231579e+00, 3.15857895e+00, 3.68484211e+00,
       4.21110526e+00, 4.73736842e+00, 5.26363158e+00, 5.78989474e+00,
       6.31615789e+00, 6.84242105e+00, 7.36868421e+00, 7.89494737e+00,
       8.42121053e+00, 8.94747368e+00, 9.47373684e+00, 1.00000000e+01]),
                         'LR__multi_class': ['ovr', 'multinomial'],
                         'LR__solver': ['lbfgs', 'liblinear', 'newton-cg',
                                        'newton-cholesky', 'sag', 'saga']})

In [195]:
print(gcv.best_params_)
print(gcv.best_score_)

{'LR__C': 2.6323157894736844, 'LR__multi_class': 'ovr', 'LR__solver': 'lbfgs'}
0.9502370779927956


In [190]:
pd_cv = pd.DataFrame(gcv.cv_results_)
print(pd_cv.shape)

(240, 16)


In [198]:
np.__version__

'1.26.4'